# Cleaning Data

This notebook is all about the meat of data science: evaluating and cleaning data. It covers a wide swath of different things and I expect it will be my most-referenced notebook. I also expect it to grow a lot with time, as I encounter different situations that require novel approaches to what is contained here. 

For now, these are the basic sections:

* Selecting/dropping, adding new columns, and renaming columns
* Dealing with missing values
* Combining and separating columns
* Grouping and Aggregation
* Value manipulation: applying, mapping
* Melting and Pivoting
* Date/time data
* Using dfply (maybe)

I'm likely to need a bunch of datasets to get things that are messed up enough for some of these things. I think the titanic dataset will do! I'll load my necessary libraries and that dataset before proceeding:

In [196]:
# Load libraries
import pandas as pd
import numpy as np
import seaborn as sns

# Load the seaborn titanic dataset
titanic = sns.load_dataset('titanic')
print(titanic.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
survived       891 non-null int64
pclass         891 non-null int64
sex            891 non-null object
age            714 non-null float64
sibsp          891 non-null int64
parch          891 non-null int64
fare           891 non-null float64
embarked       889 non-null object
class          891 non-null category
who            891 non-null object
adult_male     891 non-null bool
deck           203 non-null category
embark_town    889 non-null object
alive          891 non-null object
alone          891 non-null bool
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.6+ KB
None


Next, I'm going to do some basic operations, essentially around selecting and renaming variables

## Basic Operations

First of all, I can select a subset of values; perhaps I only care about the a certain number of columns

### Selecting Columns

There's 2 basic things here: simple selection and complex selection. In the first case, I'll just be taking things by column name or number; in the second, I'll be filtering by more complex criteria.


#### Simple Selection

Let's say I only wanted the first 3 columns...here's the way to do it

##### By Column Name

Easiest way is grabbing the names directly:

In [41]:
# Take the first 3 columns by name
titanic[['survived', 'pclass', 'sex']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 3 columns):
survived    891 non-null int64
pclass      891 non-null int64
sex         891 non-null object
dtypes: int64(2), object(1)
memory usage: 21.0+ KB


Other way to do this is by using the `.loc`:

In [42]:
# Take the first 3 columns by name
titanic.loc[:,['survived', 'pclass', 'sex']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 3 columns):
survived    891 non-null int64
pclass      891 non-null int64
sex         891 non-null object
dtypes: int64(2), object(1)
memory usage: 21.0+ KB


##### By Column Number

In [43]:
# Take the first 3 columns by number
titanic.iloc[:,0:3].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 3 columns):
survived    891 non-null int64
pclass      891 non-null int64
sex         891 non-null object
dtypes: int64(2), object(1)
memory usage: 21.0+ KB


Or take non-consecutive columns:

In [44]:
# Take the first 3 columns by number
titanic.iloc[:,[0,3,5]].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 3 columns):
survived    891 non-null int64
age         714 non-null float64
parch       891 non-null int64
dtypes: float64(1), int64(2)
memory usage: 21.0 KB


#### Selection by Filtering

This is just covering how to filter by columns, not by rows. This is basically just the `filter` method, which lets me choose things by label content (not the actual data content). 

I can simply filter by column name:

In [161]:
titanic.filter(items = ['survived','age','parch']).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 3 columns):
survived    891 non-null int64
age         891 non-null float64
parch       891 non-null int64
dtypes: float64(1), int64(2)
memory usage: 21.0 KB


Of course that's not a good reason to use it; rather, it's more useful with more flexible arguments, such as `like`, which lets me match substrings:

In [162]:
titanic.filter(like = 'class').info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 2 columns):
pclass    891 non-null int64
class     891 non-null category
dtypes: category(1), int64(1)
memory usage: 8.0 KB


I can also use regular expressions to match, such as all columns whose names start with an "s":

In [163]:
titanic.filter(regex='^s').info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 3 columns):
survived    891 non-null int64
sex         891 non-null object
sibsp       891 non-null int64
dtypes: int64(2), object(1)
memory usage: 21.0+ KB


Not much more to say for that, it's pretty straightforward; next we'll do the opposite operation, which is dropping columns

### Dropping Columns

Easiest way is probably by name; in this example, I'm going to get rid of the obnoxious duplicate variables: `embark_town`, `adult_male`, and `alive`:

In [47]:
titanic.drop(['embark_town', 'adult_male', 'alive'], axis = 'columns').info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
survived    891 non-null int64
pclass      891 non-null int64
sex         891 non-null object
age         714 non-null float64
sibsp       891 non-null int64
parch       891 non-null int64
fare        891 non-null float64
embarked    889 non-null object
class       891 non-null category
who         891 non-null object
deck        203 non-null category
alone       891 non-null bool
dtypes: bool(1), category(2), float64(2), int64(4), object(3)
memory usage: 65.8+ KB


I don't think there's a better way to do things there. Now I'll show how to add columns

### Adding Columns

There's a couple scenarios to deal with here:

* Adding 1 value (broadcasting)
* Adding 1 column
* Adding multiple columns

These are in increasing order of complexity, so let's do them 1 at a time:

#### Adding 1 value (broadcasting)

Occasionally, I find myself wanting to add 1 common value to each row; often, this is because I'm about to join or concatenate with another data frame, so I want some kind of identifier. The most straightforward way is to simply assign it as a scalar, at which point it will be 'broadcast' to all rows:

In [48]:
# Actually adds the column
titanic['broadcast'] = 'Titanic'
titanic.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone,broadcast
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False,Titanic
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False,Titanic
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True,Titanic
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False,Titanic
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True,Titanic


Now I've got a broadcast column, which could be useful later. I'm going to also show how to do the same thing using the `assign` method:

In [49]:
# Doesn't actually add in place
titanic.assign(broadcast2 = 'Titanic').head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone,broadcast,broadcast2
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False,Titanic,Titanic
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False,Titanic,Titanic
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True,Titanic,Titanic
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False,Titanic,Titanic
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True,Titanic,Titanic


The `assign` method seems excessive here, but it does let me control whether to actually add things. Observe that the dataset doesn't actually have the last column:

In [50]:
titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 16 columns):
survived       891 non-null int64
pclass         891 non-null int64
sex            891 non-null object
age            714 non-null float64
sibsp          891 non-null int64
parch          891 non-null int64
fare           891 non-null float64
embarked       889 non-null object
class          891 non-null category
who            891 non-null object
adult_male     891 non-null bool
deck           203 non-null category
embark_town    889 non-null object
alive          891 non-null object
alone          891 non-null bool
broadcast      891 non-null object
dtypes: bool(2), category(2), float64(2), int64(4), object(6)
memory usage: 87.6+ KB


So the `assign` method is more flexible, but the direct assignment is a bit faster to type out. 

Now I'll move to assigning actual columns

#### Adding 1 column

This is again easily done if I want to change the dataframe in place, which I often do:

In [51]:
# Add a column of random noise
titanic['noise'] = np.random.rand(len(titanic))
titanic.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone,broadcast,noise
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False,Titanic,0.938297
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False,Titanic,0.473655
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True,Titanic,0.965693
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False,Titanic,0.771249
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True,Titanic,0.339541


Just as before, I can use `assign` to create the column as well, but this time I won't actually add it:

In [52]:
# Add column of random noise, but don't actually save it
titanic.assign(noise2 = np.random.rand(len(titanic))).head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone,broadcast,noise,noise2
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False,Titanic,0.938297,0.443772
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False,Titanic,0.473655,0.229310
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True,Titanic,0.965693,0.603039
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False,Titanic,0.771249,0.385821
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True,Titanic,0.339541,0.741340


Again, the `assign` is more flexible and probably a better idea. I'm going to drop the added variables and then move to adding multiple columns

In [53]:
# Drop the 2 added variables
titanic.drop(['broadcast', 'noise'], axis = 'columns', inplace=True)

#### Adding Multiple Columns

This is a pain with the direct assignment; it's essentially just repeating the 1 over and over

Instead, I'll jump straight to `assign` and use it; as before, I won't save them to a new variable. If I wanted to keep it, I'd have to create a new dataframe to save the copy. 

In [54]:
# Assign 2 more columns
titanic.assign(broadcast = "Data Name",
               noise = np.random.rand(len(titanic))).head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone,broadcast,noise
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False,Data Name,0.096075
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False,Data Name,0.912215
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True,Data Name,0.286027
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False,Data Name,0.273444
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True,Data Name,0.272392


And that's it! There will be more on adding later, but that's in the context of other, more complicated operations. For now, let's move to renaming, which is super useful

### Renaming Columns

There's a couple ways to do it; first, there's the brute force way of just assigning ALL of the column names. 


#### Brute Force: Rename EVERYTHING

For this, let's take a play slice of the titanic dataframe:

In [55]:
# Take just the first 3 columns
play_slice = titanic[['survived', 'pclass', 'sex']]

Now, the brute force method just changes ALL the columns at once, so I have to do something like this:

In [56]:
play_slice.columns = ['DidNotDie', 'Fanciness', 'Gender']
play_slice.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 3 columns):
DidNotDie    891 non-null int64
Fanciness    891 non-null int64
Gender       891 non-null object
dtypes: int64(2), object(1)
memory usage: 21.0+ KB


That's a rather coarse way of doing things; if we want to be more fine about it, then we should use `rename`

#### Better Way: Use `rename`

The `rename` method allows me to supply a dictionary, where the keys are the current column names and the values are what I want to rename to. So I can do the same thing, but be more specific, in this case renaming to the original names:

In [57]:
play_slice.rename({'DidNotDie' : 'Survived',
                   'Fanciness' : 'Pclass',
                   'Gender' : 'Sex'}, axis = 'columns').info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 3 columns):
Survived    891 non-null int64
Pclass      891 non-null int64
Sex         891 non-null object
dtypes: int64(2), object(1)
memory usage: 21.0+ KB


**Important Note: I need to actually specify the axis argument, or else it'll think I mean the index instead of columns**

The best parts about this method:

* Won't automatically overwrite; have to specify `inplace` to do that
* Finer control; can just specify a subset of columns to rename
* Specificity; if a key doesn't match, it doesn't get changed

In general, using `rename` is the way to go here. And that's the last of the "basic" operations; now let's move on to missing values.

## Missing Values

There's a few different strategies here:

* Drop rows with missing values
* Fill with overall imputed value
    * Mean
    * Median
* Fill imputed value by group
* Fill with order (good for time series)
    * Forward fill
    * Backward fill

### Drop Rows

Doing this is simple:

In [58]:
print(titanic.dropna().info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 182 entries, 1 to 889
Data columns (total 15 columns):
survived       182 non-null int64
pclass         182 non-null int64
sex            182 non-null object
age            182 non-null float64
sibsp          182 non-null int64
parch          182 non-null int64
fare           182 non-null float64
embarked       182 non-null object
class          182 non-null category
who            182 non-null object
adult_male     182 non-null bool
deck           182 non-null category
embark_town    182 non-null object
alive          182 non-null object
alone          182 non-null bool
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 18.2+ KB
None


Clearly the problem here is that I lose most of the data, so this is clearly not a preferable strategy here. Better to probably impute things.

### Impute them with overall values

The most straightforward way is by using just the overall values here's filling age with the mean value:

In [59]:
# Calculate mean age
mean_age = titanic.age.mean()

# Fill with the mean age
mean_fill = titanic.age.fillna(mean_age)

# Print the info 
print(mean_fill.describe())

count    891.000000
mean      29.699118
std       13.002015
min        0.420000
25%       22.000000
50%       29.699118
75%       35.000000
max       80.000000
Name: age, dtype: float64


So now it's filled with the mean, but maybe that doesn't make sense because the mean isn't a whole number. Fill it with the median instead!

In [60]:
# Calculate mean age
median_age = titanic.age.median()

# Fill with the mean age
median_fill = titanic.age.fillna(median_age)

# Print the info 
print(median_fill.describe())

count    891.000000
mean      29.361582
std       13.019697
min        0.420000
25%       22.000000
50%       28.000000
75%       35.000000
max       80.000000
Name: age, dtype: float64


Now we have a whole number (28) that we're filling. 

### Impute them with grouped values

The one problem I think we're now running into is that we're filling in the same age for everyone. That's despite the fact that we have info on people via the 'who' field:

In [61]:
# Print it
print(titanic.who.unique())

['man' 'woman' 'child']


So what about if we fill with group-specific values? Let's do it using the median:

In [62]:
# Fill the age
median_group_fill = titanic.groupby('who')['age'].transform(lambda x: x.fillna(x.median()))
print(median_group_fill.describe())

count    891.000000
mean      29.758889
std       13.002570
min        0.420000
25%       22.000000
50%       30.000000
75%       35.000000
max       80.000000
Name: age, dtype: float64


This is probably the most accurate thing to do,as we've filled the groups individually. Great job! 

I'm going to fill like that and then drop the deck column

In [64]:
titanic = titanic.assign(age = median_group_fill).drop(['deck'], axis = 'columns')

Now I'm going to move on to how to combine and separate columns

## Combining and Separating Columns

Occasionally, it's important to combine 2 columns into 1. I'm actually going to try it using a sample data frame:

In [66]:
# Create sample data frame
sample_df = pd.DataFrame({'name': ['Sandy', 'Jenny', 'Willian', 'Juan', 'Sandy', 'Willian'],
                          'number': [1, 5, 3, 7, 1, 3],
                          'type': ['sandwich', 'meatball', 'sandwich', 'hotdog', 'hotdog', 'meatball']})
sample_df

,name,number,type
0,Sandy,1,sandwich
1,Jenny,5,meatball
2,Willian,3,sandwich
3,Juan,7,hotdog
4,Sandy,1,hotdog
5,Willian,3,meatball


Now I'll start by combining

### Combining 2 Columns

There's a couple different cases here:

1. Simple combination, no dimension change
2. Complex combination, reduction of dimensions

#### Simple Column Combinations

Let's do the simple case of putting these together; need to use the `str` tag here to change the non-string column:

In [84]:
simple_combine = sample_df.assign(combined = sample_df['name'] + '-' + sample_df['number'].map(str))
simple_combine

,name,number,type,combined
0,Sandy,1,sandwich,Sandy-1
1,Jenny,5,meatball,Jenny-5
2,Willian,3,sandwich,Willian-3
3,Juan,7,hotdog,Juan-7
4,Sandy,1,hotdog,Sandy-1
5,Willian,3,meatball,Willian-3


This type of simple combination may help for combining 2 columns, at which point we could drop the individual columns. But a more complex case exists!

#### Complex Column Combinations

Let's say we wanted to collapse each of these records into a semi-colon delimited thing. So Sandy would have type of 'sandwich;hotdog'. To do this, we need to melt:

In [83]:
complex_combine = sample_df.assign(Joined = sample_df.groupby(['name','number'], as_index=False).transform(lambda x: ';'.join(x))).drop(['type'], axis = 'columns').drop_duplicates()

complex_combine

,name,number,Joined
0,Sandy,1,sandwich;hotdog
1,Jenny,5,meatball
2,Willian,3,sandwich;meatball
3,Juan,7,hotdog


Just to recap on that, we did the following:

* Grouped by name and number to keep a person's records together
* Transformed by joining, using a lambda function and semi-colon as delimiter
* Dropped the type, so we don't have both type AND joined type
* Dropped the duplicates, so we only have the unique records

The `transform` command is super useful, as I think it's the best thing to approximate a window function. This will get its own section later, but for now, on to separating columns!

### Separating Columns

Again, there's 2 basic ways to do this:

* Simple separation (non-redundant values)
* Complex separation (redundant values)

Let's start with the simple case

#### Simple separation

We'll start with the simple combined case and separate it. It's a fairly simple operation; one way is by doing it step by step, saving as we go.

In [100]:
# Get the split into a list
simple_combine['str_split'] =simple_combine['combined'].str.split('-')

# Split off each argument into a column of its own
simple_combine['name2'] = simple_combine['str_split'].str.get(0)
simple_combine['number2'] = simple_combine['str_split'].str.get(1)
simple_combine

,name,number,type,combined,str_split,name2,number2
0,Sandy,1,sandwich,Sandy-1,"[Sandy, 1]",Sandy,1
1,Jenny,5,meatball,Jenny-5,"[Jenny, 5]",Jenny,5
2,Willian,3,sandwich,Willian-3,"[Willian, 3]",Willian,3
3,Juan,7,hotdog,Juan-7,"[Juan, 7]",Juan,7
4,Sandy,1,hotdog,Sandy-1,"[Sandy, 1]",Sandy,1
5,Willian,3,meatball,Willian-3,"[Willian, 3]",Willian,3


That's one way to do it; another way is by using the `tolist()` function:

In [102]:
simple_combine[['name3','number3']] = pd.DataFrame(simple_combine['combined'].str.split('-').tolist())
simple_combine

,name,number,type,combined,str_split,name2,number2,name3,number3
0,Sandy,1,sandwich,Sandy-1,"[Sandy, 1]",Sandy,1,Sandy,1
1,Jenny,5,meatball,Jenny-5,"[Jenny, 5]",Jenny,5,Jenny,5
2,Willian,3,sandwich,Willian-3,"[Willian, 3]",Willian,3,Willian,3
3,Juan,7,hotdog,Juan-7,"[Juan, 7]",Juan,7,Juan,7
4,Sandy,1,hotdog,Sandy-1,"[Sandy, 1]",Sandy,1,Sandy,1
5,Willian,3,meatball,Willian-3,"[Willian, 3]",Willian,3,Willian,3


This way is MUCH faster; I should do this in the future. Now I'll move to the complex splitting

#### Complex Separation

This is what to use when we want to "explode" the data. For instance, recall the complex combined data:

In [103]:
# Print the combined data
complex_combine

,name,number,Joined
0,Sandy,1,sandwich;hotdog
1,Jenny,5,meatball
2,Willian,3,sandwich;meatball
3,Juan,7,hotdog


In this case, I have redundant values, as Sandy and Willian will have to be repeated if I separate the joined column. But I can do this and return the original dataframe data; I'm going to make a function that does it:

In [104]:
def splitExpandColumn(df, col_name, delimiter = ','):
    '''Split a column of a dataframe given a delimiter'''
    
    # Split the column on commas and stack them up
    s = df[col_name].str.split(delimiter, expand=True).stack()
    # Grab the initial row numbers from the stacked values
    i = s.index.get_level_values(0)
    # Using the index, copy the rows to match the stacked values
    df2 = df.loc[i].copy()
    # Replace the comma-separated column with the split values
    df2[col_name] = s.values
    
    return df2

Now apply it:

In [105]:
splitExpandColumn(complex_combine, 'Joined', ';')

,name,number,Joined
0,Sandy,1,sandwich
0,Sandy,1,hotdog
1,Jenny,5,meatball
2,Willian,3,sandwich
2,Willian,3,meatball
3,Juan,7,hotdog


## Grouping and Aggregation: Simple Stuff

These 2 go hand in hand and they're super useful! There's a few things I want to cover here:

* Basic grouping + aggregation functions
* Multiple Aggregations
* Grouping + Filtering
* Grouping + Transforming

I'll start with the most basic, using the titanic dataset

### Basic Grouping + Aggregation 

All we have to do here is group by some variable and then do an aggregation. In this case, I'll take the mean ages by "who":

In [107]:
# Group by person type (man/woman/child) and get mean age
titanic.groupby('who')['age'].mean()

who
child     6.369518
man      32.440410
woman    31.608856
Name: age, dtype: float64

Worth noting: if I don't pull just the `age` column, I'll get the mean for every column.

There's not much too this; lots of different functions work, but I can also do some custom function if needed:

In [108]:
# Get the range of ages
titanic.groupby('who')['age'].agg(lambda x: max(x) - min(x))

who
child    14.58
man      64.00
woman    47.00
Name: age, dtype: float64

The `agg` function comes in handy here and it's even more useful (and quite necessary) for doing multiple aggregations.

### Multiple Aggregations

Sometimes I want to get multiple values, such as the mean AND standard deviation. In this case, I want to use `agg` again, but now I want to give it multiple things back. Let's also save this to a variable so I can show something else...

In [112]:
# Get mean AND standard error of the mean
grouped_stats = titanic.groupby('who')['age'].agg(['mean','sem'])
grouped_stats

,mean,sem
who,,
child,6.369518,0.519082
man,32.440410,0.491720
woman,31.608856,0.627880


Now say that I want to actually access these values; it's fairly straightforward

In [113]:
grouped_stats['mean']

who
child     6.369518
man      32.440410
woman    31.608856
Name: mean, dtype: float64

This is pretty easy and I can, as before, do custom functions. I could also take different aggregations of different variables:

In [119]:
# Do both age and survived, but do different aggregations
grouped_diff = titanic.groupby('who').agg({'age' : ['mean','median'],
                                           'survived' : ['mean', 'sem']})
grouped_diff

age         survived          
            mean median      mean       sem
who                                        
child   6.369518    5.0  0.590361  0.054307
man    32.440410   30.0  0.163873  0.015989
woman  31.608856   30.0  0.756458  0.026121

Now we can access each separately, using pretty intuitive notation on the multi-indexed columns:. To get all sub-columns under 1 heading:

In [122]:
grouped_diff['age']

,mean,median
who,,
child,6.369518,5.0
man,32.440410,30.0
woman,31.608856,30.0


To get a specific column, access it similarly:

In [123]:
grouped_diff['age']['mean']

who
child     6.369518
man      32.440410
woman    31.608856
Name: mean, dtype: float64

Now I'll move on to filtering on grouped results

### Filtered Results by Group

Sometimes, I might want to filter grouped results using the `filter` command I showed earlier. Recall the simple data I created; I'll add some more data:

In [125]:
# Print the simple one
sample_df['orders'] = [14, 21, 12, 15, 16, 11]
sample_df

,name,number,type,orders
0,Sandy,1,sandwich,14
1,Jenny,5,meatball,21
2,Willian,3,sandwich,12
3,Juan,7,hotdog,15
4,Sandy,1,hotdog,16
5,Willian,3,meatball,11


Now suppose I want only the folks with average order above 13; this will eliminate Willian:

In [128]:
sample_df.groupby('name').filter(lambda x: x['orders'].mean() > 13)

,name,number,type,orders
0,Sandy,1,sandwich,14
1,Jenny,5,meatball,21
3,Juan,7,hotdog,15
4,Sandy,1,hotdog,16


**Note: this doesn't collapse records, but it does let you filter based on grouped values**

Now I'll move on to the complex machinations of using `transform`

### Grouping + Transforming

The `transform` function lets me apply a funciton to grouped data, but keep the original shape of the data. It's similar to a window function, so I can do things like calculate group-based means for each record, but add them as a new column rather than collapsing the data.

Here's `transform` to create a columns with group-based means of orders:

In [164]:
sample_df.groupby('name')['orders'].transform(np.mean)

0    15.0
1    21.0
2    11.5
3    15.0
4    15.0
5    11.5
Name: orders, dtype: float64

I can of course use other functions, such as my range function:

In [165]:
sample_df.groupby('name')['orders'].transform(lambda x: max(x) - min(x))

0    2
1    0
2    1
3    0
4    2
5    1
Name: orders, dtype: int64

This is much faster than the alternate way of doing it, which would be:

* Group data
* Get grouped aggregate values as a data frame
* Merge with the original data to add it as a full column

Now I'll move on to a similar set of things: manipulating values with built-in functions

## Value Manipulations: Applying and Mapping

This is a grouping of several very important methods for manipulating variables in a dataframe. For this, I think I'll create some new toy data. In order, here's what each operation does:

* `apply` : apply a function over 1 direction of a data frame; means you can do means, mins, etc. in 1 direction easily
    * e.g. Apply a function across rows and save it as a new column
    * Only necessary for things that aren't data frame methods (mean, median, and others are already methods)
* `map` : similar to `apply`, but it applies the function element-wise to a *series*
    * e.g. Apply some sort of formatting change to a column to save as a new column
* `applymap` : blend of `apply` and `map`, in that it applies a function element-wise but does it on a data frame
    * e.g. Apply some sort of formatting change to the whole data frame
    
Let's tackle these 1 at a time:

### Using `apply` on Data Frames

If I want something like column-wise ranges, then `apply` works really well. Let's quickly make new data, something like trials of a measurement:

In [139]:
trials = pd.DataFrame({'sample' : [1,1,2,2,3,3,4,4],
                       'trial1' : [1,7,5,6,1,9,1,4],
                       'trial2' : [3,5,4,5,2,6,3,3],
                       'trial3' : [4,4,3,3,4,8,4,2]})
trials

,sample,trial1,trial2,trial3
0,1,1,3,4
1,1,7,5,4
2,2,5,4,3
3,2,6,5,3
4,3,1,2,4
5,3,9,6,8
6,4,1,3,4
7,4,4,3,2


Now let's apply the range over the columns (not rows):

In [140]:
# Take the ranges of the numeric columns
trials.apply(lambda x: max(x) - min(x), axis = 0)

sample    3
trial1    8
trial2    4
trial3    6
dtype: int64

If I wanted the mean of the 2 columns, I could do the same, but there's already a built-in function. 

I can also apply the same function across rows:

In [156]:
trials.apply(lambda x: max(x) - min(x), axis = 1)

0    3
1    6
2    3
3    4
4    3
5    6
6    3
7    2
dtype: int64

I can also pass additional arguments, such as this:

In [157]:
# function for adding a supplied number
def add_integer(x, num):
    return x + num

So say that I wanted to add 10 to each value, I could do that using `apply`:

In [160]:
trials.apply(add_integer, args = (10,))

,sample,trial1,trial2,trial3
0,11,11,13,14
1,11,17,15,14
2,12,15,14,13
3,12,16,15,13
4,13,11,12,14
5,13,19,16,18
6,14,11,13,14
7,14,14,13,12


I think you can use `apply` with `groupby`, but I can't find a good time to do it, so I won't. 

### Using `map` on Series

The `map` method is made to do something similar, but only on a Series:

In [169]:
trials['trial1'].map(lambda x: x + 1)

0     2
1     8
2     6
3     7
4     2
5    10
6     2
7     5
Name: trial1, dtype: int64

It's most useful with custom functions and things like that. Or for formatting:

In [172]:
trials['trial1'].map(lambda x: ('{:0.1f}'.format(x)))

0    1.0
1    7.0
2    5.0
3    6.0
4    1.0
5    9.0
6    1.0
7    4.0
Name: trial1, dtype: object

I can also use it to put together 2 series based on their values, but I won't bother here

### Using `applymap` on Data Frames

The idea with `applymap` is that we can apply a function to the whole data frame, element-by-element. This is different from `apply`, which does it by axis. So we can, e.g., format:

In [173]:
trials.applymap(lambda x: ('{:0.1f}'.format(x)))

,sample,trial1,trial2,trial3
0,1.0,1.0,3.0,4.0
1,1.0,7.0,5.0,4.0
2,2.0,5.0,4.0,3.0
3,2.0,6.0,5.0,3.0
4,3.0,1.0,2.0,4.0
5,3.0,9.0,6.0,8.0
6,4.0,1.0,3.0,4.0
7,4.0,4.0,3.0,2.0


With all this talk about reformatting, this seems as good a time as any to move to melting and pivoting

## Reshaping: Melting and Pivoting

These 2 are used to re-shape the data, either to make it into a long form (melting) or a wide form (pivoting). 

As an example, let's take this sample data on trials and reshape it:

### Melting Data

Melting the trial data will make it long, which is sometime easier for plotting (among other things). Important variables to specify:

* `id_vars` : record identifiers; won't get melted
* `value_vars` : values to melt; if it's unspecified, it'll be everything not in `id_vars`
* `var_name` : What to call the melted variables
* `value_name` : What to call the melted values

In [185]:
# Melt with samples as IDs; name the results
generic_melt = pd.melt(trials, id_vars="sample",
                       var_name = 'Trial',
                       value_name = 'Number')
generic_melt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24 entries, 0 to 23
Data columns (total 3 columns):
sample    24 non-null int64
Trial     24 non-null object
Number    24 non-null int64
dtypes: int64(2), object(1)
memory usage: 656.0+ bytes


As shown, I get 24 rows (3 different trials, 8 different samples). If I decide to specify the `value_vars`, I get something slightly different; in this case, suppose that I don't want the first trial because it's messed up:

In [183]:
# Melt with samples as IDs; name the results
pd.melt(trials, id_vars="sample",
        value_vars = ['trial2','trial3'],
        var_name = 'Trial',
        value_name = 'Number').info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16 entries, 0 to 15
Data columns (total 3 columns):
sample    16 non-null int64
Trial     16 non-null object
Number    16 non-null int64
dtypes: int64(2), object(1)
memory usage: 464.0+ bytes


By specifying this argument, I've grabbed just the columns I want. It's great! 

Now let's move to how to go back to wide from the `generic_melt` data

### Pivoting Data

This is how to get back to wide data; I will basically always use `pivot_table` as it's more flexible than the plain `pivot` method. Again, there's some things to specify:

* `index` : column(s) to use as the IDs
* `columns` : columns to pivot
* `aggfunc` : function(s) to aggregate if there's duplicate records
* `fill_value` : used to replace missing values; super useful if you're pivoting to make a heat map and you need 0's for the missing values

Because we've got duplicate samples, I'll specify the aggregation function as the median

In [188]:
generic_melt.pivot_table(index = 'sample', columns = 'Trial',
                         aggfunc = np.median)

Number              
Trial  trial1 trial2 trial3
sample                     
1         4.0    4.0    4.0
2         5.5    4.5    3.0
3         5.0    4.0    6.0
4         2.5    3.0    3.0

I think that's it for pivoting. 

## Date-Time Manipulations

Dates and times are themselves a pain in the butt. Some things I want to cover here:

* Creating a date-time column
* Separating date-time into its components

I'll add more as I think of them

### Creating a date-time column

For this, I need data with some date/time information. I'll load the `flights` dataset for this:

In [190]:
flights = sns.load_dataset('flights')
flights.head()

,year,month,passengers
0,1949,January,112
1,1949,February,118
2,1949,March,132
3,1949,April,129
4,1949,May,121


This one is particularly pesky because it's got the month as a word rather than a number. I'll show how to do the following:

* Add a "day" column that just uses the first day of each month
* Use the 3 different date columns to create 1 "Date" column
* Turn it into an actual date-time type

First 2 steps here:

In [193]:
# Add a dummy column for day
flights['day'] = 1

# Turn to a date-time
flights['Date'] = flights['year'].astype(str) + '-' + flights['month'].astype(str) + '-' + flights['day'].astype(str)
print(flights.info())
flights.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 144 entries, 0 to 143
Data columns (total 5 columns):
year          144 non-null int64
month         144 non-null category
passengers    144 non-null int64
day           144 non-null int64
Date          144 non-null object
dtypes: category(1), int64(3), object(1)
memory usage: 5.1+ KB
None


,year,month,passengers,day,Date
0,1949,January,112,1,1949-January-1
1,1949,February,118,1,1949-February-1
2,1949,March,132,1,1949-March-1
3,1949,April,129,1,1949-April-1
4,1949,May,121,1,1949-May-1


Now I'll actually do the conversion

In [195]:
flights['Date'] = pd.to_datetime(flights['Date'], format = '%Y-%B-%d')
print(flights.info())
flights.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 144 entries, 0 to 143
Data columns (total 5 columns):
year          144 non-null int64
month         144 non-null category
passengers    144 non-null int64
day           144 non-null int64
Date          144 non-null datetime64[ns]
dtypes: category(1), datetime64[ns](1), int64(3)
memory usage: 5.1 KB
None


,year,month,passengers,day,Date
0,1949,January,112,1,1949-01-01
1,1949,February,118,1,1949-02-01
2,1949,March,132,1,1949-03-01
3,1949,April,129,1,1949-04-01
4,1949,May,121,1,1949-05-01


That's all there is to it; the `format` argument lets me change what I'm looking for, but I can always look that up. 

### Separating a Date-Time

Sometimes I find that I want to pull out the month or year, or something like that. It might be good for grouping or something like that. 